In [31]:
import pandas as pd
import numpy as np
import os

# Load CSV (with encoding fix)
df = pd.read_csv(
    "C:/Users/Dell/Desktop/new dp/DP/db/csv/25datacsv.csv",
    encoding="latin1"   # try "ISO-8859-1" or "cp1252" if needed
)

In [33]:
df.head(10)

,Row,Actual Product,Segment,DP FLAG,MotherBatchNo,Ip Width,Ip Thick,Mother Ip Wt,Order Tdc,Op Batch No,...,PDOTr,PDOPor,PDOSpeedSetupFurn,Schd Line No,NRI,RA_CODE,Surface_Roughness_Min,Surface_Roughness_Max,Unnamed: 139,Unnamed: 140
0,1,GI,WG,Y,2414VJZ000,1540,0.500,11.07,ZAP130,2414VJZ000,...,1.0,2.0,85.0,NaN,NaN,D1,0.7,1.6,NaN,NaN
1,2,GI,WG,Y,24178S0000,1410,0.477,26.35,ZAP221,24178S1000,...,NaN,NaN,NaN,NaN,NaN,D4,0.8,1.2,NaN,NaN
2,3,GI,WG,N,24178S0000,1410,0.477,26.35,ZAP221,24178S2000,...,NaN,NaN,NaN,NaN,NaN,D4,0.8,1.2,NaN,NaN
3,4,GI,WG,Y,2422XK0000,1200,0.970,23.38,ZST935,2422XK1000,...,NaN,NaN,NaN,NaN,NaN,D2,0.8,1.4,NaN,NaN
4,5,GI,WG,Y,2422XKZ000,1200,0.970,20.57,ZST935,2422XKZ000,...,2.0,0.0,98.0,1.0,R1,D2,0.8,1.4,NaN,NaN
5,6,GI,GE,Y,2425VH0000,900,0.779,14.18,ZGN110,2425VH1000,...,NaN,NaN,NaN,NaN,NaN,B2,0.0,0.6,NaN,NaN
6,7,GI,GE,Y,2425VH0000,900,0.779,14.18,ZGN110,2425VH2000,...,NaN,NaN,NaN,NaN,NaN,B2,0.0,0.6,NaN,NaN
7,8,GI,GE,Y,2425WJ0000,900,0.779,15.22,ZGN110,2425WJ1000,...,1.0,2.0,105.0,NaN,NaN,B2,0.0,0.6,NaN,NaN
8,9,GI,GE,Y,2425WJ0000,900,0.779,15.22,ZGN110,2425WJ2000,...,1.0,2.0,105.0,NaN,NaN,B2,0.0,0.6,NaN,NaN
9,10,GI,AUTO,Y,24308R0000,1116,0.589,19.90,ZPK001,24308R1000,...,NaN,NaN,NaN,NaN,NaN,B2,0.0,0.6,NaN,NaN


In [93]:
# ---- SEGMENT mapping ----
def get_segment(val):
    if str(val).startswith("ZAP"):
        return "Appliance"
    elif str(val).startswith(("ZST", "ZGN")):
        return "Retail"
    elif str(val).startswith("ZTU"):
        return "P&T"
    elif str(val).startswith("ZPL"):
        return "Panel"
    elif str(val).startswith("ZEX"):
        return "export"
    elif str(val).startswith("ZPK"):
        return "GI Packing"
    elif str(val).startswith("APK"):
        return "Appliance Packing"
    else:
        return "Other"

df["Segment"] = df["Order Tdc"].apply(get_segment)

# ---- Ensure Process Duration rounded ----
if "Process Duration(in min)" in df.columns:
    df["Process Duration(in min)"] = df["Process Duration(in min)"].round(0)
else:
    df["Process Duration(in min)"] = 0   # fallback if missing

# ---- Area = Ip Width × Total Length ----
if "Area" not in df.columns:
    if {"Ip Width", "Total Length"}.issubset(df.columns):
        df["Area"] = ((df["Ip Width"] * df["Total Length"]) / 1_000_000).round(2)
    else:
        df["Area"] = np.nan

# ---- Zinc = (Total Zn/AlZn Coating) × Area ----
if "Zinc" not in df.columns:
    if {"Total Zn/AlZn Coating", "Area"}.issubset(df.columns):
        df["Zinc"] = (df["Total Zn/AlZn Coating"] * df["Area"]).round(3)
    else:
        df["Zinc"] = np.nan

# ---- CRFH thickness ----
if {"Prop Ip Wt", "Ip Width", "Total Length"}.issubset(df.columns):
    df["CRFH thickness(mm)"] = (
        (df["Prop Ip Wt"] * 1000) /
        (7.850 * (df["Ip Width"] * df["Total Length"]) / 1000)
    ).round(3)
else:
    df["CRFH thickness(mm)"] = np.nan

# ---- GP thickness ----
def calc_gp_thickness(row):
    if row["Actual Product"] == "GI":
        factor = 7140
    elif row["Actual Product"] in ["GL", "PPGL"]:
        factor = 3750
    elif row["Actual Product"] == "ZM":
        factor = 6850
    else:
        factor = np.nan

    if pd.notna(factor):
        return round((row["Total Zn/AlZn Coating"] / factor) + row["CRFH thickness(mm)"], 4)
    else:
        return np.nan

df["GP thickness(mm)"] = df.apply(calc_gp_thickness, axis=1)

# ---- Speed ----
if {"Total Length", "Process Duration(in min)"}.issubset(df.columns):
    df["Speed(mpm)"] = (df["Total Length"] / df["Process Duration(in min)"]).round(3)
else:
    df["Speed(mpm)"] = np.nan

# ---- Productivity ----
if {"O/P Wt", "Process Duration(in min)"}.issubset(df.columns):
    df["Productivity(TPH)"] = ((df["O/P Wt"] / df["Process Duration(in min)"]) * 60).round(3)
else:
    df["Productivity(TPH)"] = np.nan


In [95]:
# ---- Final Columns ----
final_columns = [
    "Op Batch No", "Actual Product", "Actual Tdc", "Segment",
    "Prop Ip Wt", "O/P Wt", "Total Length", "Area", "Zinc",
    "Process Duration(in min)", "CRFH thickness(mm)", "GP thickness(mm)",
    "Total Zn/AlZn Coating", "Op Width", "Speed(mpm)", "Productivity(TPH)"
]

available_columns = [col for col in final_columns if col in df.columns]
df_final = df[available_columns]

In [97]:
df_final.head(10)

,Op Batch No,Actual Product,Segment,Prop Ip Wt,O/P Wt,Total Length,Area,Zinc,Process Duration(in min),CRFH thickness(mm),GP thickness(mm),Total Zn/AlZn Coating,Op Width,Speed(mpm),Productivity(TPH)
0,2414VJZ000,GI,Appliance,11.070,11.228,1825.500,2.8113,0.000,19.0,NaN,NaN,130.514,1540,96.079,35.457
1,24178S1000,GI,Appliance,15.659,14.940,2690.014,3.7929,0.000,10.0,NaN,NaN,128.000,1410,269.001,89.640
2,24178S2000,GI,Appliance,10.691,10.200,1836.556,2.5895,0.000,10.0,NaN,NaN,128.000,1410,183.656,61.200
3,2422XK1000,GI,Retail,2.810,2.720,288.747,0.3465,0.000,19.0,NaN,NaN,520.000,1200,15.197,8.589
4,2422XKZ000,GI,Retail,20.570,21.795,2265.700,2.7188,0.001,24.0,NaN,NaN,529.766,1200,94.404,54.488
5,2425VH1000,GI,Retail,8.051,8.081,1426.589,1.2839,0.000,18.0,NaN,NaN,88.000,900,79.255,26.937
6,2425VH2000,GI,Retail,6.129,6.152,1086.051,0.9774,0.000,13.0,NaN,NaN,88.000,900,83.542,28.394
7,2425WJ1000,GI,Retail,8.040,8.080,1464.800,1.3183,0.000,13.0,NaN,NaN,93.842,900,112.677,37.292
8,2425WJ2000,GI,Retail,7.180,7.215,1306.400,1.1758,0.000,11.0,NaN,NaN,88.084,900,118.764,39.355
9,24308R1000,GI,GI Packing,5.760,5.740,1092.010,1.2187,0.000,12.0,NaN,NaN,88.000,1116,91.001,28.700


KeyError: 'Ip Width'

In [107]:
print(df[["Ip Width", "Total Length", "area1"]].head(10))

   Ip Width  Total Length    area1
0      1540      1825.500  2811.27
1      1410      2690.014  3792.92
2      1410      1836.556  2589.54
3      1200       288.747   346.50
4      1200      2265.700  2718.84
5       900      1426.589  1283.93
6       900      1086.051   977.45
7       900      1464.800  1318.32
8       900      1306.400  1175.76
9      1116      1092.010  1218.68


In [123]:
# ---- Save Output ----
df_final.to_csv("filtered_output.csv", index=False)

print("✅ Final CSV saved as filtered_output.csv")

✅ Final CSV saved as filtered_output.csv


In [137]:
# detect_encoding.py
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

file_path = 'C:/Users/Dell/Desktop/order/filtered_output.csv'
encoding = detect_encoding(file_path)
print(f"Detected encoding: {encoding}")

Detected encoding: ascii


In [120]:
# simplified_model_training.py (updated)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pickle
import re
import warnings
warnings.filterwarnings('ignore')

# Load your dataset with the correct encoding
file_path = 'C:/Users/Dell/Desktop/order/filtered_output.csv'

# Try different encodings
try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_path, encoding='latin1')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='ISO-8859-1')
        print("Used ISO-8859-1 encoding")

print(f"Dataset shape: {df.shape}")
print(f"Dataset columns: {df.columns.tolist()}")

# Define the specific features we need for the web app
# We'll use Product Type, Thickness, TDC, and Zinc
target_column = "Productivity(TPH)"

# Check for target column
if target_column not in df.columns:
    print(f"Error: Target column '{target_column}' not found in dataset")
    # Try to find similar target column
    similar_targets = [c for c in df.columns if 'productivity' in c.lower() or 'tph' in c.lower()]
    if similar_targets:
        target_column = similar_targets[0]
        print(f"Using alternative target: '{target_column}'")
    else:
        print("Error: No suitable target column found")
        exit()

# Function to extract numerical value from TDC
def extract_tdc_value(tdc_string):
    if pd.isna(tdc_string) or tdc_string == '':
        return 0
    
    tdc_str = str(tdc_string)
    numbers = re.findall(r'\d+\.?\d*', tdc_str)
    
    if numbers:
        return float(numbers[0])
    else:
        return 0

# Create a new column with numerical TDC values
df['TDC_Value'] = df['Order Tdc'].apply(extract_tdc_value)

# Look for thickness columns
thickness_columns = [col for col in df.columns if 'thickness' in col.lower()]
if thickness_columns:
    thickness_column = thickness_columns[0]
    print(f"Using '{thickness_column}' as thickness feature")
else:
    print("Warning: No thickness column found, using default value")
    # Add a default thickness column
    df['Thickness'] = 1.0
    thickness_column = 'Thickness'

# Look for zinc columns
zinc_columns = [col for col in df.columns if 'zinc' in col.lower() and 'coating' not in col.lower()]
if zinc_columns:
    zinc_column = zinc_columns[0]
    print(f"Using '{zinc_column}' as zinc feature")
else:
    print("Warning: No zinc column found, using default value")
    # Add a default zinc column
    df['Zinc'] = 130.0
    zinc_column = 'Zinc'

# Look for product type columns
product_columns = [col for col in df.columns if 'product' in col.lower() and 'type' in col.lower()]
if product_columns:
    product_column = product_columns[0]
    print(f"Using '{product_column}' as product type feature")
else:
    print("Warning: No product type column found, using default value")
    # Add a default product type column
    df['Product_Type'] = 'GI'
    product_column = 'Product_Type'

# Define our features
required_features = [product_column, 'TDC_Value', zinc_column, thickness_column]

print(f"Using features: {required_features}")
print(f"Using target: {target_column}")

# Filter the dataset
data = df[required_features + [target_column]].copy()

print(f"Data shape before cleaning: {data.shape}")
print(f"Missing values before cleaning:")
print(data.isnull().sum())

# Handle missing values
for col in required_features:
    if data[col].dtype == 'object':  # For categorical columns
        if data[col].isnull().any():
            print(f"Filling missing values in {col} with 'Unknown'")
            data[col].fillna('Unknown', inplace=True)
    else:  # For numeric columns
        if data[col].isnull().any():
            print(f"Filling missing values in {col} with median")
            data[col].fillna(data[col].median(), inplace=True)

# Fill target column missing values
if data[target_column].isnull().any():
    print(f"Filling missing values in {target_column} with median")
    data[target_column].fillna(data[target_column].median(), inplace=True)

print(f"Data shape after handling missing values: {data.shape}")

# Check for infinite values in target column
print(f"Checking for infinite values in target column '{target_column}'")
infinite_mask = np.isinf(data[target_column])
if infinite_mask.any():
    print(f"Found {infinite_mask.sum()} infinite values in target column")
    # Remove rows with infinite values
    data = data[~infinite_mask]
    print(f"Removed {infinite_mask.sum()} rows with infinite values")

# Check for extremely large values in target column
if len(data) > 0:
    Q1 = data[target_column].quantile(0.25)
    Q3 = data[target_column].quantile(0.75)
    IQR = Q3 - Q1
    outlier_mask = (data[target_column] < (Q1 - 3 * IQR)) | (data[target_column] > (Q3 + 3 * IQR))
    
    if outlier_mask.any():
        print(f"Found {outlier_mask.sum()} extreme outliers in target column")
        # Cap them at the 99th and 1st percentiles
        upper_limit = data[target_column].quantile(0.99)
        lower_limit = data[target_column].quantile(0.01)
        data[target_column] = np.clip(data[target_column], lower_limit, upper_limit)
        print(f"Capped target values to range [{lower_limit:.2f}, {upper_limit:.2f}]")

# Check if we have enough data
if len(data) < 10:
    print(f"Error: Not enough data ({len(data)} rows)")
    exit()

# Encode categorical variable (Product Type)
le = LabelEncoder()
data[product_column] = data[product_column].astype(str)
le.fit(data[product_column])
data[product_column] = le.transform(data[product_column])
print(f"Encoded '{product_column}' with {len(le.classes_)} categories")

# Split the data
X = data[required_features]
y = data[target_column]

# Check for any remaining issues
print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"X dtypes:\n{X.dtypes}")
print(f"Target statistics - Min: {y.min():.2f}, Max: {y.max():.2f}, Mean: {y.mean():.2f}")

# Check if y has any infinite or extremely large values
if np.isinf(y).any() or y.isnull().any():
    print("Warning: Target variable still contains invalid values")
    # Remove rows with invalid target values
    valid_mask = ~(np.isinf(y) | y.isnull())
    X = X[valid_mask]
    y = y[valid_mask]
    print(f"Removed invalid target values, new shape: X={X.shape}, y={y.shape}")

if len(X) == 0:
    print("Error: No valid data remaining after cleaning")
    exit()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Check target variable statistics
print(f"\nTarget variable '{target_column}' statistics:")
print(f"  Min: {y_train.min():.2f}")
print(f"  Max: {y_train.max():.2f}")
print(f"  Mean: {y_train.mean():.2f}")
print(f"  Std: {y_train.std():.2f}")

# Train the model
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
print("\nModel Evaluation:")
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R2 Score: {r2:.4f}")

# Calculate percentage error
if y_test.mean() > 0:
    percentage_error = (mae / y_test.mean()) * 100
    print(f"Mean Absolute Percentage Error: {percentage_error:.2f}%")
else:
    print("Mean Absolute Percentage Error: Cannot calculate (mean is zero)")

# Save the model and preprocessing objects
model_data = {
    'model': model,
    'scaler': scaler,
    'label_encoder': le,
    'features': required_features,
    'target': target_column,
    'tdc_extractor': extract_tdc_value,
    'product_column': product_column
}

with open('productivity_model.pkl', 'wb') as f:
    pickle.dump(model_data, f)

print("Model saved as productivity_model.pkl")

# Feature importance analysis
print("\nFeature Importance:")
feature_importance = pd.DataFrame({
    'feature': required_features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance)

# Test the model with sample data
print("\nSample prediction from test set:")
if len(X_test) > 0:
    sample_idx = X_test.index[0]
    sample_input = X.loc[sample_idx]
    actual_value = y.loc[sample_idx]

    sample_scaled = scaler.transform(sample_input.values.reshape(1, -1))
    predicted_value = model.predict(sample_scaled)[0]

    print(f"Input features:")
    for feature, value in sample_input.items():
        print(f"  {feature}: {value}")

    print(f"Actual {target_column}: {actual_value:.2f}")
    print(f"Predicted {target_column}: {predicted_value:.2f}")
    print(f"Error: {abs(actual_value - predicted_value):.2f}")

# Check for overfitting
y_train_pred = model.predict(X_train_scaled)
train_r2 = r2_score(y_train, y_train_pred)
print(f"\nTraining R2 Score: {train_r2:.4f}")

if train_r2 - r2 > 0.2:
    print("Warning: Model may be overfitting")
else:
    print("Model generalization appears good")

Dataset shape: (16315, 16)
Dataset columns: ['Op Batch No', 'Actual Product', 'Order Tdc', 'Segment', 'Prop Ip Wt', 'O/P Wt', 'Total Length', 'Area', 'Zinc', 'Process Duration(in min)', 'CRFH thickness(mm)', 'GP thickness(mm)', 'Total Zn/AlZn Coating', 'Op Width', 'Speed(mpm)', 'Productivity(TPH)']
Using 'CRFH thickness(mm)' as thickness feature
Using 'Zinc' as zinc feature
Using features: ['Product_Type', 'TDC_Value', 'Zinc', 'CRFH thickness(mm)']
Using target: Productivity(TPH)
Data shape before cleaning: (16315, 5)
Missing values before cleaning:
Product_Type          0
TDC_Value             0
Zinc                  0
CRFH thickness(mm)    0
Productivity(TPH)     0
dtype: int64
Data shape after handling missing values: (16315, 5)
Checking for infinite values in target column 'Productivity(TPH)'
Found 21 infinite values in target column
Removed 21 rows with infinite values
Found 111 extreme outliers in target column
Capped target values to range [14.14, 126.28]
Encoded 'Product_Type' 

In [122]:
import pandas as pd
import numpy as np

# Load CSV (with encoding fix)
df = pd.read_csv(
    "C:/Users/Dell/Desktop/new dp/DP/db/csv/25datacsv.csv",
    encoding="latin1"
)

# ---- SEGMENT mapping ----
def get_segment(val):
    if str(val).startswith("ZAP"):
        return "Appliance"
    elif str(val).startswith(("ZST", "ZGN")):
        return "Retail"
    elif str(val).startswith("ZTU"):
        return "P&T"
    elif str(val).startswith("ZPL"):
        return "Panel"
    elif str(val).startswith("ZEX"):
        return "export"
    else:
        return "Other"

df["Segment"] = df["Order Tdc"].apply(get_segment)

# ---- Ensure Process Duration rounded ----
if "Process Duration(in min)" in df.columns:
    df["Process Duration(in min)"] = df["Process Duration(in min)"].round(0)
else:
    df["Process Duration(in min)"] = 0

# ---- Area = Ip Width × Total Length (then divide by 1000) ----
if "Area" not in df.columns:
    if {"Ip Width", "Total Length"}.issubset(df.columns):
        # Calculate area and then divide by 1000
        df["Area"] = (df["Ip Width"] * df["Total Length"] / 1000).round(2)
        print("Area calculation: (Ip Width × Total Length) / 1000 = Area")
    else:
        df["Area"] = np.nan

# ---- Zinc Calculation ----
if "Zinc" not in df.columns:
    if {"Total Zn/AlZn Coating", "Area"}.issubset(df.columns):
        # Since Area is already divided by 1000, adjust zinc calculation
        # Original: Zinc = (Coating × Area) / 1,000,000,000
        # Now: Zinc = (Coating × Area) / 1,000,000 (since we already divided Area by 1000)
        df["Zinc"] = (df["Total Zn/AlZn Coating"] * df["Area"] / 1000000).round(3)
        print("Zinc calculation: (Coating g/m² × Area) / 1,000,000 = Zinc (kg)")
    else:
        df["Zinc"] = np.nan

# ---- CRFH thickness ----
if {"Prop Ip Wt", "Ip Width", "Total Length"}.issubset(df.columns):
    # Use the modified area for thickness calculation
    df["CRFH thickness(mm)"] = (
        (df["Prop Ip Wt"] * 1000) /  # Convert tons to kg
        (7.850 * df["Area"] * 1000)  # Adjust for the area division
    ).round(3)
else:
    df["CRFH thickness(mm)"] = np.nan

# ---- GP thickness ----
def calc_gp_thickness(row):
    if row["Actual Product"] == "GI":
        factor = 7140
    elif row["Actual Product"] in ["GL", "PPGL"]:
        factor = 3750
    elif row["Actual Product"] == "ZM":
        factor = 6850
    else:
        factor = np.nan

    if pd.notna(factor):
        return round((row["Total Zn/AlZn Coating"] / factor) + row["CRFH thickness(mm)"], 4)
    else:
        return np.nan

df["GP thickness(mm)"] = df.apply(calc_gp_thickness, axis=1)

# ---- Speed ----
if {"Total Length", "Process Duration(in min)"}.issubset(df.columns):
    df["Speed(mpm)"] = (df["Total Length"] / df["Process Duration(in min)"]).round(3)
else:
    df["Speed(mpm)"] = np.nan

# ---- Productivity ----
if {"O/P Wt", "Process Duration(in min)"}.issubset(df.columns):
    df["Productivity(TPH)"] = ((df["O/P Wt"] / df["Process Duration(in min)"]) * 60).round(3)
else:
    df["Productivity(TPH)"] = np.nan

# ---- Final Columns ----
final_columns = [
    "Op Batch No", "Actual Product", "Order Tdc", "Segment",
    "Prop Ip Wt", "O/P Wt", "Total Length", "Area", "Zinc",
    "Process Duration(in min)", "CRFH thickness(mm)", "GP thickness(mm)",
    "Total Zn/AlZn Coating", "Op Width", "Speed(mpm)", "Productivity(TPH)"
]

available_columns = [col for col in final_columns if col in df.columns]
df_final = df[available_columns]

# Check results
print(f"Area range: {df_final['Area'].min():.2f} to {df_final['Area'].max():.2f}")
print(f"Zinc range: {df_final['Zinc'].min():.3f} to {df_final['Zinc'].max():.3f} kg")

# Show sample values
print("\nSample values:")
for i in range(min(5, len(df_final))):
    coating = df_final['Total Zn/AlZn Coating'].iloc[i] if 'Total Zn/AlZn Coating' in df_final.columns else 'N/A'
    area = df_final['Area'].iloc[i] if 'Area' in df_final.columns else 'N/A'
    zinc = df_final['Zinc'].iloc[i] if 'Zinc' in df_final.columns else 'N/A'
    print(f"Row {i}: Coating={coating}, Area={area}, Zinc={zinc}")

# ---- Save Output ----
df_final.to_csv("filtered_output.csv", index=False)
print("✅ Final CSV saved as filtered_output.csv")

Area calculation: (Ip Width × Total Length) / 1000 = Area
Zinc calculation: (Coating g/m² × Area) / 1,000,000 = Zinc (kg)
Area range: 0.50 to 10233.43
Zinc range: 0.000 to 3.741 kg

Sample values:
Row 0: Coating=130.514, Area=2811.27, Zinc=0.367
Row 1: Coating=128.0, Area=3792.92, Zinc=0.485
Row 2: Coating=128.0, Area=2589.54, Zinc=0.331
Row 3: Coating=520.0, Area=346.5, Zinc=0.18
Row 4: Coating=529.766, Area=2718.84, Zinc=1.44
✅ Final CSV saved as filtered_output.csv
